In [1]:
import simulacra.star
import simulacra.tellurics
from simulacra.star import PhoenixModel

import random
import numpy as np

import astropy.io.fits
import astropy.time as at

import astropy.units as u
import astropy.coordinates as coord
import astropy.constants as const

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysynphot/locations.py:46: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


<h1>Simulacra: An Introduction to Simulating Spectrograph Data</h1>
This package should be used to simulate spectrographs by creating a star with a given flux, various transmission models (gas cell and tellurics), and a detector. Then simulating the detector at given start times for an exposure time.

$$ SA t_{exp} \frac{erg}{cm^3s} \frac{A_{telescope}}{A_{ste}} * d \lambda = N $$

In [2]:
ra, dec = np.random.uniform(0,360) * u.degree, np.random.uniform(0,80) * u.degree
obs = 'APO'
loc = coord.EarthLocation.of_site(obs)
target = coord.SkyCoord(ra,dec,frame='icrs')

Functions from the star module can be used to select times to view a given star from some observatory.

In [ ]:
tstart = at.Time('2020-01-01T08:10:00.123456789',format='isot',scale='utc')
tend   = tstart + 365 * u.day
night_grid = simulacra.star.get_night_grid(loc,tstart,tend,steps_per_night=10)
possible_times, airmass = simulacra.star.get_realistic_times(target,loc,night_grid)

In [ ]:
epoches = 2

Now we selected some random sample of these to observe at and the airmasses at those times

In [ ]:
obs_ints = random.sample(range(len(airmass)),epoches)
obs_times, obs_airmass = possible_times[obs_ints], airmass[obs_ints]

<h2>Tellurics Model</h2>
The tellurics model requires these airmasses at the time of observation. However each of the pressure, temperatures, and humidities can be set by the user after initialization. If a single value is passed that is used for every epoch. Or you can pass it an array of quantities of size equal to the number of epoches.

In [ ]:
wave_min = 590*u.nm
wave_max = 592*u.nm
tellurics_model = simulacra.tellurics.TelFitModel(wave_min,wave_max,loc)

Define some atmospheric parameters for the tellurics. These can either be constant over all time or an array with the same length as the number of epoches.

In [ ]:
tellurics_model.pressure    = 875 * u.hPa
tellurics_model.temperature = 300 * u.Kelvin
tellurics_model.humidity    = 50.0

<h2>Star Model</h2>
Here we define the star model with some temperature, distance, logg, and companion parameters. The logg, T, z, and alpha parameters must correspond to an appropriate atmosphere model from the PHOENIX libraray online. Then also give it some companion parameters that could affect its velocity. This is what we will be trying to find use jabble.

In [ ]:
logg = 1.0
T    = 4800
z    = 1.0
alpha= 0.4
amplitude = 2 * u.m/u.s
period    = 65 * u.day
stellar_model = PhoenixModel(alpha,z,T,logg,target,amplitude,period)

In [ ]:
from simulacra.detector import Detector, spacing_from_res

<h2>Detector</h2>
Here we define our detector giving it an aperature area, resolution, dark current, read noise, and ccd efficiency. All of these can be except area can be given as an array of the same size as the wave_grid (eg. if the detector has varying resolution or noise levels)

In [ ]:
resolution = 130_000
area = 10 *u.m**2
exp_times = 8 * np.ones(epoches)* u.minute 
dark_current = 100/u.s
read_noise   = 100
ccd_eff      = 0.99

delta_x = spacing_from_res(resolution)
x_grid = np.arange(np.log(wave_min.to(u.Angstrom).value),np.log(wave_max.to(u.Angstrom).value),delta_x)
wave_grid = np.exp(x_grid) * u.Angstrom

detector = Detector(stellar_model,resolution,loc,area,wave_grid,dark_current,read_noise,ccd_eff)

In [ ]:
detector.add_model(tellurics_model)

<h2>Gas Cell</h2>
Optionally, add the gas cell to the detector for simulations of the Keck HiRES spectrograph.

In [ ]:
from simulacra.gascell import GasCellModel
gascell_model = GasCellModel('data/gascell/keck_fts_inUse.idl')
detector.add_model(gascell_model)

<h2>Simulator</h2>
Now comes the bulk of the work, run the simulation with the given transmission models, star, detector at the given times for some exposure times.

In [ ]:
data = detector.simulate(obs_times,exp_times)

Save file to pickle jar

In [ ]:
filename = 'out/datatest4.pkl'
data.to_pickle(filename)

In [ ]:
import matplotlib.pyplot as plt
def normalize(y,constant):
    return y/constant

In [ ]:
constant = np.mean(data['theory']['interpolated']['total']['flux'])
fig, ax = plt.subplots(figsize=(20,8))
plt.xlim((5900),(5910))
plt.ylim(-.001,.001)
# data.plot_flux(ax,0,['data','flux'],['data','wave'],pargs=simulacra.dataset.data_plot_settings,xy=False)
data.plot_flux(ax,0,['theory','interpolated','star','flux'],['theory','interpolated','total','wave'],xy=False,pargs={**simulacra.dataset.star_settings,**simulacra.dataset.interpolated_settings},normalize=normalize,nargs=[constant])
# data.plot_flux(ax,0,['theory','interpolated','TelFitModel','flux'],['theory','interpolated','total','wave'],xy=False,pargs={**simulacra.dataset.tellurics_settings,**simulacra.dataset.interpolated_settings})
# data.plot_flux(ax,0,['theory','interpolated','GasCellModel','flux'],['theory','interpolated','total','wave'],xy=False,pargs={**simulacra.dataset.gas_settings,**simulacra.dataset.interpolated_settings})
data.plot_flux(ax,0,['theory','interpolated','total','flux'],['theory','interpolated','total','wave'],xy=False,pargs={'color':'gray',**simulacra.dataset.interpolated_settings},normalize=normalize,nargs=[constant])
# data.plot_flux(ax,0,['theory','lsf','flux'],['theory','interpolated','total','wave'],pargs={'color':'pink',**simulacra.dataset.interpolated_settings},normalize=normalize,nargs=[constant])
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(data['theory']['star']['wave'],data['theory']['star']['flux'][0,:])
plt.xlabel(data['theory']['star']['wave'].unit)
plt.ylabel(data['theory']['star']['flux'].to(u.ph).unit)
plt.show()

In [ ]:
from simulacra.dataset import print_keys
print_keys(data)

In [ ]:
fig, ax = plt.subplots(figsize=(20,8))
data.plot_rvs(ax)